In [5]:
import os
os.environ['CUDA_LAUNCH_BLOCKING'] = "0"

import random
import torch
import torch.nn as NN
from torch.nn import functional as F
import matplotlib.pyplot as plt

with open('data/messages.txt', 'r', encoding='utf-8') as f:
    text = f.read()
chars = sorted(list(set(text)))
i2c = { i:ch for i, ch in enumerate(chars) }
c2i = { ch[1]:i for i, ch in enumerate(i2c.items())}
N = len(chars)
LEN = len(text)
encode = lambda s: [c2i[c] for c in s]
decode = lambda l: ''.join([i2c[i] for i in l])
text_enc = encode(text)

CTX_SZ = 256 #8
xs, ys = [], []
for s in range(LEN - CTX_SZ):
    xs.append(text_enc[s:s+CTX_SZ])
    ys.append(text_enc[s+1:s+CTX_SZ+1])

tmp = list(zip(xs, ys))
random.shuffle(tmp)
xs, ys = zip(*tmp)
xs, ys = list(xs), list(ys)

n = int(0.9 * LEN)
xs_trn, ys_trn = torch.tensor(xs[:n]), torch.tensor(ys[:n])
xs_val, ys_val = torch.tensor(xs[n:]), torch.tensor(ys[n:])

device = 'cuda' if torch.cuda.is_available() else 'cpu'
xs_trn, ys_trn = xs_trn.to(device), ys_trn.to(device)
xs_val, ys_val = xs_val.to(device), ys_val.to(device)

In [6]:
class SelfAttentionHead(NN.Module):
    def __init__(self, head_sz, n_emb, ctx_sz, dropout):
        super().__init__()
        self.keys = NN.Linear(n_emb, head_sz, bias=False)
        self.queries = NN.Linear(n_emb, head_sz, bias=False)
        self.values = NN.Linear(n_emb, head_sz, bias=False)
        self.register_buffer('tril', torch.tril(torch.ones(ctx_sz, ctx_sz)))
        self.dropout = NN.Dropout(dropout)
    def forward(self, x):
        B, T, C = x.shape
        k = self.keys(x) #    (B,T,C)
        q = self.queries(x) # (B,T,C)
        weights = q @ k.transpose(-2, -1) * C**-0.5 # (B,T,T)
        weights = weights.masked_fill(self.tril[:T, :T] == 0, float('-inf')) # (B,T,T)
        weights = F.softmax(weights, dim=-1) # (B,T,T)
        weights = self.dropout(weights)
        v = self.values(x) # (B,T,C)
        out = weights @ v # (B,T,C)
        return out

class SelfAttentionMultiHead(NN.Module):
    def __init__(self, n_heads, head_sz, n_emb, ctx_sz, dropout):
        super().__init__()
        self.heads = NN.ModuleList([SelfAttentionHead(head_sz, n_emb, ctx_sz, dropout) for _ in range(n_heads)])
        self.proj = NN.Linear(n_emb, n_emb)
        self.dropout = NN.Dropout(dropout)
    def forward(self, x):
        out = torch.cat([h(x) for h in self.heads], dim=-1)
        out = self.dropout(self.proj(out))
        return out

class FeedForward(NN.Module):
    def __init__(self, n_emb, multiplier, dropout):
        super().__init__()
        self.net = NN.Sequential(
            NN.Linear(n_emb, n_emb * multiplier),
            NN.ReLU(),
            NN.Linear(multiplier * n_emb, n_emb),
            NN.Dropout(dropout)
        )
    def forward(self, x):
        return self.net(x)

class TransformerBlock(NN.Module):
    def __init__(self, n_heads, n_emb, ctx_sz, dropout):
        super().__init__()
        head_sz = n_emb // n_heads
        self.sa = SelfAttentionMultiHead(n_heads, head_sz, n_emb, ctx_sz, dropout)
        self.ffwd = FeedForward(n_emb, 4, dropout)
        self.ln1 = NN.LayerNorm(n_emb)
        self.ln2 = NN.LayerNorm(n_emb)
    def forward(self, x):
        x = x + self.sa(self.ln1(x))
        x = x + self.ffwd(self.ln2(x))
        return x

class TransformerLanguageModel(NN.Module):
    def __init__(self, n_blocks, n_heads, voc_sz, n_emb, ctx_sz, dropout):
        super().__init__()
        self.ctx_sz = ctx_sz
        self.tok_emb_table = NN.Embedding(voc_sz, n_emb)
        self.pos_emb_table = NN.Embedding(ctx_sz, n_emb)
        self.blocks = NN.Sequential(*(
            ([TransformerBlock(n_heads, n_emb, ctx_sz, dropout)] * n_blocks) +
            [NN.LayerNorm(n_emb)]
        ))
        self.lm_head = NN.Linear(n_emb, voc_sz)
    def forward(self, idx, targets=None):
        B, T = idx.shape        
        tok_emb = self.tok_emb_table(idx)
        pos_emb = self.pos_emb_table(torch.arange(T, device=device))
        x = tok_emb + pos_emb
        x = self.blocks(x)
        logits = self.lm_head(x)
        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B * T, C)
            loss = F.cross_entropy(logits, targets.view(B * T))        
        return logits, loss
    def generate(self, idx, max_new_tok):
        for _ in range(max_new_tok):
            idx_crop = idx[:, -self.ctx_sz:]
            logits, _ = self(idx_crop)
            probs = F.softmax(logits[:, -1, :], dim=1)
            idx_nxt = torch.multinomial(probs, num_samples=1)
            idx = torch.cat((idx, idx_nxt), dim=1)
        return idx

In [7]:
N_EMB = 384 #32
N_HEADS = 6
N_BLOCKS = 6
EVAL_INT = 1000
model = TransformerLanguageModel(N_BLOCKS, N_HEADS, N, N_EMB, CTX_SZ, 0.2).to(device)
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-4)

@torch.no_grad()
def est_loss(model, d):
    out = {}
    model.eval()
    for splt in ['trn', 'val']:
        lossi = torch.zeros(EVAL_INT)
        for k in range(EVAL_INT):
            bix = torch.randint(d[splt][0].shape[0] - 1, (BATCH_SZ,)).to(device)
            _, loss = model(d[splt][0][bix], d[splt][1][bix])
            lossi[k] = loss.item()
        out[splt] = lossi.mean()
    model.train()
    return out

BATCH_SZ = 64 #32
lossi = {'trn':[], 'val':[]}
for i in range(15000):
    xb, yb = xs_trn[i*BATCH_SZ : (i+1)*BATCH_SZ], ys_trn[i*BATCH_SZ : (i+1)*BATCH_SZ]
    logits, loss = model(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()
    if i % EVAL_INT == 0:
        est = est_loss(model, {'trn': [xs_trn, ys_trn], 'val': [xs_val, ys_val]})
        lossi['trn'].append(est['trn'])
        lossi['val'].append(est['val'])
        print(est['trn'])
plt.plot(lossi['trn'])
plt.plot(lossi['val'])

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39


KeyboardInterrupt: 

In [56]:
print(decode(model.generate(torch.zeros((1,1), dtype=torch.long).to(device), max_new_tok=1000)[0].tolist())) 

	<КАРТИНКА>
	этотка со своём нет как надо беспоку
ВАНЯ:	Да, кого может фигорея его лишкова?
ТИМА:	Уж)
ВАНЯ:	угадай)
	<ЦИТАТА:	сбер только сгереть?>
	heartaple
	в 0
	тебя фраза
	с машах с ними не всё мы как в такоже вторнё наопломанове и стакает атмпачка и андри - та и зависите сошо быть
ВАСЯ:	да из стретьё оставили и понедели светят?
	полезных их признаков, он выехал приду
ТИМА:	это за хорбыл болтька
	а у тебя у видел - в том, даже обрабоны, видишь проигнал
	процент, должно быть будти
ВАСЯ:	меня теперь угрозильно, к чесмодели там дураками
ТИМА:	надекал
ВАНЯ:	а посмотрим прашлывать)
	с койтоном ндот
	и если есть глопали в деловей мести
	бьюда обучали окно новые леживуют дороге гомна
	<ЦИТАТА:прыганды не занает)>
	нам, там мне помнишь выдобзовывательств Языков ос делали, хмужников себе будет погравитель отвекали:  . А предпроверил, что объявление эту штыку, и могу рассадятые фотографа, уходить сажу не звидимо туто штём добрая
	что опелил, что это синитер если этим_ени)
ВАНЯ:	ну, да эти ш

In [52]:
print(decode(model.generate(torch.tensor([encode("МАША:\tсырный")], dtype=torch.long).to(device), max_new_tok=1000)[0].tolist())) 

МАША:	сырный порт:
	Насики стара поздановате нет, испорили сервь>
	Вася на нормально) выглядена!!! (
СТЁПА:	нет там вета, как стати
	не не так "за доровеции!
	Его Сочьи политься - нибе
СТЁПА:	первая, ренда влия?
ТИМА:	зато подтавиван момент имень инктески натфильм канате на странели инотянские фотографи в стои кино приез-двет камета идея отклетовая такие и принестива
	E ука z))))
СТЁПА:	тогда да потир отказал
	актёт, даже не радили на должен, которым картом и пусти этих проходимо)
ВАНЯ:	да, информацийна оказали на не ижу)
СТЁПА:	тоже того от сколым их професов
ТИМА:	жуки тется прифрадиван за гоны с судяманный доминками сарианалам и без, непривизование о незаки называть модете выбор политательно силения)
	смотре не пойде изидеи в жизни, пложу драки не позже только трепирает и и Её
СТЁПА:	да...друга, и балаконец на такой наприпор парашив нос на отку веснове
	ина и всех ищут идео и друг она нового
	но датти)
	может так искать отправь проднепрессов, задошная при должанов ном друзь нора игр